In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import pandas as pd
from energyPlus.weather.weather import plotCZMap
import numpy as np

In [ ]:
dfFull = pd.read_csv('dfFull.csv')
flowStatsMI = pd.read_csv('LES/flowStatsMI.csv')
roomVentilationMI = pd.read_csv('LES/roomVentilationMI.csv')

In [ ]:
def snapToAngle(angle, snapAngle):
    return snapAngle * round(angle / snapAngle)

In [ ]:
dfFull['WDVent'] = dfFull['WDVent'].apply(lambda x: snapToAngle(x, 45))
# dfFull['WSTVent'] = dfFull['TintMinusTref']* dfFull['WSVent'] # Now written in the csv

counts = dfFull.groupby(["Climate Zone", "WDVent"])["WSVent"].sum() / dfFull.groupby(["Climate Zone", "WDVent"])["WSVent"].mean()
qoi = 'WSTVent'
# qoi = 'WSVent'
WDZones = dfFull.groupby(["Climate Zone", "nVent", "WDVent"])[qoi].sum()
# WDZones *= 1 / dfFull.shape[0]
WDZones

In [ ]:
groups = ["roomType", "delT", "AofA"]
dfLES = roomVentilationMI.groupby(groups)["mean-sn_prod(abs(u))-Norm"].mean()
dfLESHouse = roomVentilationMI.groupby(["delT", "AofA"])["mean-sn_prod(abs(u))-Norm"].mean()
# dfLES.loc[:,:] = 1

In [ ]:
def getOptimalAlignment(bee: pd.Series, rose:pd.Series, allowFlip=True):
    AResolution = 45
    AOptimal = 0
    maxIntegral = 0
    if allowFlip:
        flips = [1, -1]
    else:
        flips = [1]
    for A in range(0, 360, AResolution):
        for flip in flips:
            beeCopy = bee.copy()
            roseCopy = rose.copy()
            roseCopy.index = (360 + flip * roseCopy.index) % 360
            beeCopy.index = (beeCopy.index + A) % 360
            integral = (beeCopy[roseCopy.index] * roseCopy).sum()
            if integral > maxIntegral:
                maxIntegral = integral
                if A == 0:
                    A = 360
                AOptimal = A * flip
    return AOptimal, maxIntegral

getOptimalAlignment(dfLES[("corner", 0)], WDZones[(13, 0)], allowFlip=True)

In [ ]:
optimalVent = {}
optimalVentHouse = {}
for delT in dfLES.index.levels[1].values:
    nVent = int(delT == 5)
    for zone in WDZones.index.levels[0].values:
        optimalVentHouse[(delT, zone)] = getOptimalAlignment(dfLESHouse[delT], WDZones[(zone, nVent)])
        for room in dfLES.index.levels[0].values:
            optimalVent[(room, delT, zone)] = getOptimalAlignment(dfLES[(room, delT)], WDZones[(zone, nVent)])

optimalVent = pd.DataFrame(optimalVent).T
optimalVentHouse = pd.DataFrame(optimalVentHouse).T
for optV in [optimalVent, optimalVentHouse]:
    optV.columns = ["A", "ventAverage"]
    optV["Total Days"] = dfFull["Total Days"][0] #normalizing for plotting purposes
optimalVent.index.names = ["roomType", "delT", "Climate Zone"]
optimalVentHouse.index.names = ["delT", "Climate Zone"]
display(optimalVent)
display(optimalVentHouse)

In [ ]:
delT = 0
roomAndTVentAll = pd.DataFrame()
for i, room in enumerate(optimalVent.index.levels[0].values):
    print(room)
    roomAndTVent = optimalVent.loc[(room, delT)].copy()
    # roomAndTVent["A_rel"] = np.sign(roomAndTVent["A"]) * ((abs(roomAndTVent["A"]) - abs(roomAndTVent["A"].loc[ref_zone])) % 360)
    roomAndTVent = roomAndTVent.reset_index()
    roomAndTVentAll = roomAndTVentAll.add(roomAndTVent, fill_value=0)
    plotCZMap(roomAndTVent, weights='ventAverage', colormap='YlGnBu', vmax = 2750)
    roomAndTVent["Total Days"] = 365 * 16
    plotCZMap(roomAndTVent, weights='A', vmin=-360, vmax=360, colormap='Spectral')

roomAndTVentAll /= (i + 1)
plotCZMap(roomAndTVentAll, weights='ventAverage', colormap='YlGnBu', vmax = 1800)

In [ ]:
delT = 0
TVent = optimalVentHouse.loc[delT].copy()
# TVent["A_rel"] = np.sign(TVent["A"]) * ((abs(TVent["A"]) - abs(TVent["A"].loc[ref_zone])) % 360)
TVent = TVent.reset_index()
plotCZMap(TVent, weights='ventAverage', colormap='YlGnBu', vmax=1800)
TVent["Total Days"] = 365 * 16
plotCZMap(TVent, weights='A', vmin=-360, vmax=360, colormap='Spectral')